## RAG
> Retriever-Augmented Generation

: LLM 이 응답을 생성할 때 학습 데이터 소스 외부의 신뢰할 수 있는 지식 베이스를 참조할 수 있도록 하는 방법

In [1]:
# LangSmith Configure
from dotenv import load_dotenv

load_dotenv()

True

In [1]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader, PyPDFLoader, JSONLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.embeddings import HuggingFaceEmbeddings

from langchain_community.chat_models import ChatOllama
from langchain_core.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.callbacks.manager import CallbackManager


> Langchain

: Langchain은 통합된 인터페이스를 통해 언어 모델, 프롬프트, 외부 데이터 소스 등을 연결하여 복잡한 자연어 처리 태스크를 수행하도록 하는 Library

### Data 전처리

> 통합형 Loader 인터페이스

- 다양한 loader 도구를 활용하여 loader 객체를 생성한다.
    - Web page
    - pdf
    - json 
    - csv
    - etc
- 생성된 loader 객체의 `load()` 함수는 전체 문서를 로드하여 반환한다.
- 생성된 loader 객체의 `load_and_split()` 함수는 문서를 페이지로 쪼개어 반환한 결과이다.
- 반환된 document 구조는 `page_content` 와 `metadata` 속성 값을 포함한다.
    - `page_content`는 문서의 내용
    - `metadata`는 파일명, 페이지 번호, 그 밖에 유용한 메타정보를 포함한다.



#### 다양한 문서를 불러오는 방법

In [2]:
# WebBaseLoader
web_loader = WebBaseLoader(
    web_path=("https://news.sbs.co.kr/news/endPage.do?news_id=N1007601801&plink=TOTAL&cooper=SBSNEWSSEARCH",),
    bs_kwargs= dict(
        parse_only=bs4.SoupStrainer(
            'div',
            attrs={"class": ["text_area",]},
        )
    ),
)
docs = web_loader.load()
print(f"문서의 수: {len(docs)}")
print(docs)

문서의 수: 1
[Document(page_content="\n  <앵커> \n  \n 한 지방교육청이 박람회에 쓸 주제가를 공모했습니다.\xa0그런데, 1등으로 뽑은 곡이 알고 보니 AI가 만든 노래였는데요, 이 공모전에 심사위원으로 참여했던 유명한 작곡가도 이런 사실을 '전혀 몰랐다'고 말하기도 했습니다. \n  \n 홍영재 기자가 보도합니다. \n  \n <기자> \n  \n 다음 달 전남 여수에서 열리는 '글로컬 미래교육 박람회' 주제가로 선정된 곡입니다. \n  \n [제목 '세상에 소리쳐! 글로컬!' : 변화의 바람 우리의 힘을 깨우네. 글로컬 교육이여 미래를 여는 열쇠 가자~! 내가 원하는 미래 글로컬 네가 원하는 미래 글로컬] \n  \n 응모작 12곡 가운데 희망찬 가사, 밝은 악풍이 높게 평가돼 한 초등학교 교사가 만든 이 노래가 주제가로 선정됐습니다. \n  \n 선정 사실을 통보하는 과정에서 새로운 사실이 드러났습니다. \n  \n [전남교육청 관계자 : 이거를 살펴보는 과정에서 선생님께서도 AI 곡이라고 이제 말씀을 주셔 가지고 그래서 나중에 전달이 됐습니다.] \n  \n AI 서비스에 문자 명령 입력을 수차례 거쳐 만들었다는 겁니다. \n  \n 작곡도, 노래도 모두 AI였던 겁니다. \n  \n 심사위원으로 참여한 유명 작곡가 김형석 씨는 '제법 수작이었다'며 당혹했습니다. \n  \n [김형석/작곡가 : 전혀 몰랐어요. 전혀 몰랐고. '어 잠깐만 그럼 이걸 1위로 줘야 되나?' 이제 그런 퀘스천 마크(물음표)가 생겼고….] \n  \n 주최 측은 AI를 사용하지 말라는 조건이 없었고, 미래세대를 위한 박람회의 주제와 부합하다며 주제곡으로 최종 선정했습니다. \n  \n AI가 일상뿐 아니라 예술계 전반으로 확산하면서 창작의 정의에 대한 논란은 거셉니다. \n  \n 독일의 한 사진작가는 국제사진전에 일부러 AI가 만든 이미지로 응모한 뒤 1위에 선정되자 수상을 거부하기도 했습니다. \n  \n 그림, 음악 등 여러 예술 분야에 이

In [3]:
# PyPDFLoader
pdf_loader = PyPDFLoader('1706.03762v7.pdf')
split_docs = pdf_loader.load_and_split()
print(f"문서의 수: {len(split_docs)}")
print(split_docs[0])


문서의 수: 16
page_content='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing 

In [4]:
# CSV
csv_loader = CSVLoader(file_path='output.csv')
data = csv_loader.load()
print(f"행의 수: {len(data)}")


행의 수: 9272


In [5]:
# JSON
json_loader = JSONLoader(
    file_path='example_2.json',
    jq_schema=".",
    text_content=False,
)
data = json_loader.load()
print(data)

[Document(page_content='{"quiz": {"sport": {"q1": {"question": "Which one is correct team name in NBA?", "options": ["New York Bulls", "Los Angeles Kings", "Golden State Warriros", "Huston Rocket"], "answer": "Huston Rocket"}}, "maths": {"q1": {"question": "5 + 7 = ?", "options": ["10", "11", "12", "13"], "answer": "12"}, "q2": {"question": "12 - 8 = ?", "options": ["1", "2", "3", "4"], "answer": "4"}}}}', metadata={'source': '/root/rag_prac/example_2.json', 'seq_num': 1})]


#### Langchain Interface 전체 Process
- `Indexing`: load, split embed, store
- `RAG`: prompt, retriever, generation

In [7]:
loader = WebBaseLoader(
    web_path=("https://news.sbs.co.kr/news/endPage.do?news_id=N1007601801&plink=TOTAL&cooper=SBSNEWSSEARCH",),
    bs_kwargs= dict(
        parse_only=bs4.SoupStrainer(
            'div',
            attrs={"class": ["text_area",]},
        )
    ),
)

docs = loader.load()

text_spliter = RecursiveCharacterTextSplitter(
    chunk_size=300, chunk_overlap=50,
)

splits = text_spliter.split_documents(docs)

model_name = "jhgan/ko-sroberta-multitask"
EMBEDDING = HuggingFaceEmbeddings(model_name=model_name)

vector_store = FAISS.from_documents(
    documents=splits, embedding=EMBEDDING
)

retriever = vector_store.as_retriever()

prompt = hub.pull("rlm/rag-prompt")

llm = ChatOllama(
    model='yanolja-10.8B:latest',
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [8]:
chain.invoke(
    "최근 김형석 아티스트가 AI 로 만든 곡을 1위로 뽑았다는데, 자세히 이야기해줘."
)

최근 김형석 아티스트는 AI가 만든 곡을 한 지방교육청에서 개최한 주제가 공모전의 1위로 선정했습니다. 심사위원인 김형석은 AI 사용 사실을 몰라서 놀랐으며, 이는 예술과 창작의 정의에 대한 질문을 제기하였습니다. 주최 측은 미래 세대를 위한 박람회의 주제를 반영하기 위해 이 노래를 최종적으로 채택했습니다.

'최근 김형석 아티스트는 AI가 만든 곡을 한 지방교육청에서 개최한 주제가 공모전의 1위로 선정했습니다. 심사위원인 김형석은 AI 사용 사실을 몰라서 놀랐으며, 이는 예술과 창작의 정의에 대한 질문을 제기하였습니다. 주최 측은 미래 세대를 위한 박람회의 주제를 반영하기 위해 이 노래를 최종적으로 채택했습니다.'